https://zenodo.org/records/5727800

# imports

In [632]:
!pip install neurokit2 --quiet
!pip install neurokit2 pyhrv --quiet
!pip install pyhrv peakutils --quiet
!pip uninstall -y biosppy pyhrv
!pip install biosppy==0.6.1 pyhrv==0.4.0 --quiet


Found existing installation: biosppy 0.6.1
Uninstalling biosppy-0.6.1:
  Successfully uninstalled biosppy-0.6.1
Found existing installation: pyhrv 0.4.0
Uninstalling pyhrv-0.4.0:
  Successfully uninstalled pyhrv-0.4.0


In [633]:
import neurokit2 as nk
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
from scipy.stats import zscore
from scipy.integrate import trapezoid
from scipy.interpolate import interp1d
from scipy.signal import welch
from tensorflow import keras
from pyhrv.frequency_domain import welch_psd
from pyhrv.nonlinear import sample_entropy

# upload data from drive

In [565]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [566]:
from google.colab import drive
drive.mount('/content/drive')
import os
import keras
import joblib

project_dir = '/content/drive/MyDrive/Colab Notebooks/project/'
trained_models_dir = os.path.join(project_dir, 'trained models/')

# Check if the file exists before attempting to load
hrv_model_path = trained_models_dir+"stress models/keras/stress_from_hrv_model_up.keras"

s_from_hrv_model = keras.models.load_model(trained_models_dir+"stress models/keras/stress_model_up.keras")
s_from_step_model = keras.models.load_model(trained_models_dir+"stress models/keras/stress_from_steps_model.keras")
s_from_ox_model = keras.models.load_model(trained_models_dir+"stress models/keras/stress_from_sleep_and_ox_model.keras")

scaler_steps = joblib.load(trained_models_dir+"stress models/pkl/steps_scaler.pkl")
scaler_oxsleep = joblib.load(trained_models_dir+"stress models/pkl/ox_and_sleep_scaler.pkl")
scaler_hrv = joblib.load(trained_models_dir+'stress models/pkl/hrv_scaler.pkl')

encoder_hrv = joblib.load(trained_models_dir+'stress models/hrv_encoder.pkl')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [567]:
#s_from_hrv_model.summary()

In [568]:
'''import os
import pandas as pd
import numpy as np
import re

def load_ecg_segments(base_dir):
    records = []

    for fname in sorted(os.listdir(base_dir)):
        if fname.endswith(".csv") and fname.startswith("ecg_sub"):
            match = re.match(r"ecg_sub(\d+)_seg(\d+)\.csv", fname)
            if match:
                sub = int(match.group(1))
                seg = int(match.group(2))
                file_path = os.path.join(base_dir, fname)

                try:
                    df = pd.read_csv(file_path, header=None)
                    # Get the first column as the signal
                    raw_data = df.iloc[:, 0].dropna().astype(np.float32).tolist()

                    records.append({
                        "sub": sub,
                        "seg": seg,
                        "raw data": raw_data
                    })
                except Exception as e:
                    print(f"⚠️ Error reading {fname}: {e}")

    return pd.DataFrame(records)'''

'import os\nimport pandas as pd\nimport numpy as np\nimport re\n\ndef load_ecg_segments(base_dir):\n    records = []\n\n    for fname in sorted(os.listdir(base_dir)):\n        if fname.endswith(".csv") and fname.startswith("ecg_sub"):\n            match = re.match(r"ecg_sub(\\d+)_seg(\\d+)\\.csv", fname)\n            if match:\n                sub = int(match.group(1))\n                seg = int(match.group(2))\n                file_path = os.path.join(base_dir, fname)\n\n                try:\n                    df = pd.read_csv(file_path, header=None)\n                    # Get the first column as the signal\n                    raw_data = df.iloc[:, 0].dropna().astype(np.float32).tolist()\n\n                    records.append({\n                        "sub": sub,\n                        "seg": seg,\n                        "raw data": raw_data\n                    })\n                except Exception as e:\n                    print(f"⚠️ Error reading {fname}: {e}")\n\n    return 

In [569]:
import ast
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/'
save_dir = project_dir + 'Integration and simulation/'

base_dir = project_dir+'Integration and simulation/ecg_for_check'

#df1 = load_ecg_segments(base_dir)
original_df = pd.read_csv(save_dir+'ecg_segments_loaded.csv')
df = original_df.copy()
df["raw data"] = df["raw data"].apply(ast.literal_eval)

# Sanity check
#df.head()
#print(df.columns)

'''# prompt: lets save into a directory in drive df

save_dir = project_dir + 'Integration and simulation/'
#os.makedirs(save_dir, exist_ok=True)
df.to_csv(save_dir + 'ecg_segments_loaded.csv', index=False)'''

"# prompt: lets save into a directory in drive df\n\nsave_dir = project_dir + 'Integration and simulation/'\n#os.makedirs(save_dir, exist_ok=True)\ndf.to_csv(save_dir + 'ecg_segments_loaded.csv', index=False)"

# Model configuration sanity cheack

In [601]:
test_df = pd.read_csv("/content/drive/MyDrive/project/test.csv")

In [573]:
'''import joblib
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model


model = s_from_hrv_model
scaler = scaler_hrv
encoder = encoder_hrv

row = test_df.iloc[3]  # change index if you like

# Drop label/id columns to isolate features
features = row.drop(["datasetId", "condition"]).values.reshape(1, -1)

# STEP 4: Scale and reshape
scaled = scaler.transform(features).reshape(1, 1, 34)

# STEP 5: Predict
prediction = model.predict(scaled)
predicted_index = np.argmax(prediction)
predicted_label = encoder.categories_[0][predicted_index]

# STEP 6: Show result
print("Prediction probabilities:", prediction)
print("Predicted class:", predicted_label)
print("True class:", row['condition'])'''

'import joblib\nimport numpy as np\nimport pandas as pd\nfrom tensorflow.keras.models import load_model\n\n\nmodel = s_from_hrv_model\nscaler = scaler_hrv\nencoder = encoder_hrv\n\nrow = test_df.iloc[3]  # change index if you like\n\n# Drop label/id columns to isolate features\nfeatures = row.drop(["datasetId", "condition"]).values.reshape(1, -1)\n\n# STEP 4: Scale and reshape\nscaled = scaler.transform(features).reshape(1, 1, 34)\n\n# STEP 5: Predict\nprediction = model.predict(scaled)\npredicted_index = np.argmax(prediction)\npredicted_label = encoder.categories_[0][predicted_index]\n\n# STEP 6: Show result\nprint("Prediction probabilities:", prediction)\nprint("Predicted class:", predicted_label)\nprint("True class:", row[\'condition\'])'

# extract features


In [634]:
df

,sub,seg,raw data
0,10,1,"[-2.0, -2.0, -1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
1,10,2,"[-18.0, -22.0, -26.0, -25.0, -10.0, 45.0, 127...."
2,10,3,"[0.0, 1.0, 5.0, 12.0, 16.0, 16.0, 17.0, 18.0, ..."
3,10,4,"[18.0, 15.0, 14.0, 14.0, 17.0, 19.0, 13.0, 7.0..."
4,10,5,"[83.0, 111.0, 130.0, 126.0, 102.0, 51.0, -13.0..."
...,...,...,...
94,8,5,"[-37.0, -44.0, -51.0, -55.0, -61.0, -62.0, -56..."
95,9,1,"[32.0, 20.0, 10.0, 4.0, 2.0, 2.0, 3.0, 3.0, 5...."
96,9,2,"[52.0, 56.0, 63.0, 71.0, 78.0, 82.0, 84.0, 85...."
97,9,3,"[28.0, 31.0, 34.0, 32.0, 31.0, 33.0, 39.0, 46...."


In [581]:
def extract_hrv_features(ecg_signal, sampling_rate=250):#, datasetId=None):
    # 1. R-peak detection using NeuroKit2
    _, rpeaks_info = nk.ecg_peaks(ecg_signal, sampling_rate=sampling_rate)
    rpeaks = rpeaks_info["ECG_R_Peaks"]  # indices of R-peaks in the signal

    # 2. Compute RR intervals (ms) from consecutive R-peak indices
    rr_intervals = np.diff(rpeaks) / sampling_rate * 1000.0  # convert index difference to milliseconds

    # 3. Remove RR outliers (<300 ms or >2000 ms) to clean artifacts
    rr_intervals = rr_intervals[(rr_intervals >= 300) & (rr_intervals <= 2000)]

    # If not enough intervals remain, you may handle separately (not covered here for brevity)

    # 4. Time-domain features
    # Basic statistics on RR intervals
    mean_rr   = float(np.mean(rr_intervals))
    median_rr = float(np.median(rr_intervals))
    sdrr      = float(np.std(rr_intervals, ddof=0))            # SDNN: std of RR
    # Successive differences of RR intervals
    diff_rr   = np.diff(rr_intervals)
    if diff_rr.size > 0:
        rmssd = float(np.sqrt(np.mean(diff_rr**2)))            # RMSSD
        sdsd  = float(np.std(diff_rr, ddof=0))                 # SDSD
    else:
        rmssd = 0.0
        sdsd  = 0.0
    sdrr_rmssd = float(sdrr / rmssd) if rmssd != 0 else 0.0    # ratio SDNN/RMSSD
    # Heart rate (beats per minute) from mean RR (ms)
    hr = float(0.0 if mean_rr == 0 else (60000.0 / mean_rr))
    # pNN25 and pNN50 (percentage of absolute differences above 25 ms and 50 ms)
    abs_diff_rr = np.abs(diff_rr)
    pnn25 = float(100.0 * np.sum(abs_diff_rr > 25) / abs_diff_rr.size) if abs_diff_rr.size > 0 else 0.0
    pnn50 = float(100.0 * np.sum(abs_diff_rr > 50) / abs_diff_rr.size) if abs_diff_rr.size > 0 else 0.0

    # 5. Poincaré plot features (SD1, SD2) and distribution shape (kurtosis, skewness)
    sd1 = float(sdsd / np.sqrt(2.0))
    sd2 = float(np.sqrt(2.0 * (sdrr**2) - 0.5 * (sdsd**2))) if sdrr >= 0 and sdsd >= 0 else 0.0
    # Kurtosis and skewness of RR intervals
    if rr_intervals.size > 0:
        # Using scipy.stats if available; otherwise, compute manually
        # Here we compute population statistics (Fisher's definition for kurtosis)
        m_rr = mean_rr
        std_rr = np.std(rr_intervals, ddof=0)
        if std_rr == 0:
            kurt = 0.0
            skew = 0.0
        else:
            z_rr = (rr_intervals - m_rr) / std_rr
            kurt = float(np.mean(z_rr**4) - 3)  # excess kurtosis
            skew = float(np.mean(z_rr**3))
    else:
        kurt = 0.0
        skew = 0.0

    # 6. Relative RR interval features (ratio of successive RR intervals)
    if rr_intervals.size > 1:
        rel_rr = rr_intervals[1:] / rr_intervals[:-1]  # ratio of each interval to the previous
    else:
        rel_rr = np.array([])
    if rel_rr.size > 0:
        mean_rel_rr   = float(np.mean(rel_rr))
        median_rel_rr = float(np.median(rel_rr))
        sdrr_rel_rr   = float(np.std(rel_rr, ddof=0))
    else:
        mean_rel_rr = 0.0
        median_rel_rr = 0.0
        sdrr_rel_rr = 0.0
    # Differences of relative RR series
    diff_rel_rr = np.diff(rel_rr)
    if diff_rel_rr.size > 0:
        rmssd_rel_rr = float(np.sqrt(np.mean(diff_rel_rr**2)))
        sdsd_rel_rr  = float(np.std(diff_rel_rr, ddof=0))
    else:
        rmssd_rel_rr = 0.0
        sdsd_rel_rr  = 0.0
    sdrr_rmssd_rel_rr = float(sdrr_rel_rr / rmssd_rel_rr) if rmssd_rel_rr != 0 else 0.0
    # Kurtosis and skewness of relative RR series
    if rel_rr.size > 0:
        m_rel = mean_rel_rr
        std_rel = np.std(rel_rr, ddof=0)
        if std_rel == 0:
            kurt_rel_rr = 0.0
            skew_rel_rr = 0.0
        else:
            z_rel = (rel_rr - m_rel) / std_rel
            kurt_rel_rr = float(np.mean(z_rel**4) - 3)
            skew_rel_rr = float(np.mean(z_rel**3))
    else:
        kurt_rel_rr = 0.0
        skew_rel_rr = 0.0

    # 7. Frequency-domain features using Welch's PSD (pyHRV)
    # Compute PSD and frequency-domain metrics
    fd_res = welch_psd(nni=rr_intervals, show=False, show_param=False)
    # Absolute powers [ms^2] in VLF, LF, HF bands
    vlf, lf, hf = fd_res['fft_abs']
    # Relative powers [% of total power]
    vlf_pct, lf_pct, hf_pct = fd_res['fft_rel']
    # Normalized units (LF, HF)
    lf_nu, hf_nu = fd_res['fft_norm']
    # Total power and LF/HF ratio
    tp = fd_res['fft_total']
    lf_hf = fd_res['fft_ratio']
    hf_lf = float(hf / lf) if lf != 0 else float('inf')

    # 8. Non-linear features: Sample Entropy and Higuchi Fractal Dimension
    sampen_result = sample_entropy(rr_intervals)
    sampen_val = sampen_result[0]
    #print(f"Extracted sampen_val from rr_intervals: {sampen_val}, Type: {type(sampen_val)}")

    # Higuchi fractal dimension using NeuroKit2
    #hfd_val, _ = nk.fractal_higuchi(rr_intervals, show=False)
    hfd_tuple = nk.fractal_higuchi(rr_intervals, show=False)
    hfd_val = hfd_tuple[0]
    #print(f"Extracted hfd_val from tuple: {hfd_tuple}, Type: {type(hfd_tuple)}")


    # 9. Compile all features into a list in the specified order
    features = [
        mean_rr, median_rr, sdrr, rmssd, sdsd, sdrr_rmssd, hr, pnn25, pnn50,
        sd1, sd2, kurt, skew,
        mean_rel_rr, median_rel_rr, sdrr_rel_rr, rmssd_rel_rr, sdsd_rel_rr, sdrr_rmssd_rel_rr, kurt_rel_rr, skew_rel_rr,
        float(vlf), float(vlf_pct), float(lf), float(lf_pct), float(lf_nu), float(hf), float(hf_pct), float(hf_nu),
        float(tp), float(lf_hf), float(hf_lf),
        float(sampen_val), float(hfd_val)#,
        #datasetId
    ]
    return features

# predictions

In [749]:
row = df[(df["sub"] == 10) & (df["seg"] == 3)]
#print(row)
raw = row["raw data"].iloc[0]
#print(raw)

row_features = extract_hrv_features(raw)
print(row_features)
hrv_array = np.array(row_features, dtype=np.float32).reshape(1, 1, -1)
predict_stress_from_hrv(hrv_array)
#print(hrv_array)
#a = generate_features_for_row(10, 3, df)
#print(a)
#hrv_scaled = scaler_hrv.transform(hrv_array)  # shape still (1, 34)
#print(hrv_scaled)

[482.15384615384613, 348.0, 172.6721420468665, 221.16533467783302, 221.0089430511518, 0.7807378235761696, 124.4416081684748, 44.73684210526316, 42.10526315789474, 156.27692233434092, 187.64024308184224, -1.1143153029330353, 0.3618320545638605, 1.130276295824566, 1.0, 0.5380869855651034, 0.8500611947928585, 0.8500611276598038, 0.6329979404555969, 0.4092956884790393, 1.0753020626224088, 751.7372124710936, 4.084751625215988, 1886.9757399430223, 10.253353289693356, 10.690013802215152, 15764.785753124543, 85.66189508509066, 89.30998619778484, 18403.49870553866, 0.11969561588041425, 8.354524872482232, 0.7651206801850345, 1.6726697796790704]
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step


'No Stress'

In [576]:
'''import numpy as np
import pandas as pd

def generate_features_for_row(sub, seg, df, sampling_rate=250):
    """
    Given sub and seg identifiers, compute HRV features and add 3 additional random features.

    Parameters:
        sub (int): subject ID
        seg (int): segment ID
        df (DataFrame): your main DataFrame that contains the raw data
        sampling_rate (int): ECG signal sampling rate (default 250Hz)

    Returns:
        List of 37 feature values: 34 HRV + 3 random physiological
    """
    row = df[(df["sub"] == sub) & (df["seg"] == seg)]

    if row.empty:
        raise ValueError(f"No data found for sub={sub}, seg={seg}")

    raw_signal = row["raw data"].iloc[0]

    try:
        hrv_features = extract_hrv_features(raw_signal, sampling_rate=sampling_rate)
    except Exception as e:
        print(f"❌ Failed to extract HRV features for sub={sub}, seg={seg}: {e}")
        hrv_features = [np.nan] * 34

    # Generate random physiological features
    blood_oxygen = round(np.random.uniform(90.0, 99.0), 1)
    sleeping_hours = round(np.random.uniform(0.0, 9.0), 1)
    step_count = np.random.randint(0, 2000)

    return hrv_features[:34] + [blood_oxygen, sleeping_hours, step_count]'''

In [762]:
'''def predict_stress_from_all_models(sub, seg, df):
    # Generate all 34 HRV features + 3 physiologic inputs
    full_features = generate_features_for_row(sub, seg, df)
    if full_features is None or len(full_features) != 37:
        print(f"⚠️ Feature extraction failed for sub={sub}, seg={seg}")
        return None

    # Split into parts
    hrv_features = np.array(full_features[:34], dtype=np.float32).reshape(1, -1)
    print(hrv_features, type(hrv_features))
    ox = full_features[34]
    sleep = full_features[35]
    steps = full_features[36]
    print(ox, sleep, steps)
    print(type(ox), type(sleep), type(steps))

    # Prepare inputs for each model
    #x_hrv = hrv_features.reshape(1, 1, 34)
    # First reshape to (1, 34)
    #hrv_features = np.array(full_features[:34], dtype=np.float32).reshape(1, -1)

    # Apply the scaler you used during training
    #hrv_features_scaled = scaler_hrv.transform(hrv_features)  # <- MUST match training

    # Reshape for model input: (1, 1, 34)

    # Predict
    #probs_hrv = s_from_hrv_model.predict(x_hrv)

    #x_hrv = hrv_features_scaled.transform(1, 1, 34)
    x_steps = scaler_steps.transform([[steps]])       # shape (1, 1)
    x_ox = scaler_oxsleep.transform([[ox, sleep]])

    # Predict class probabilities
    #probs_hrv = s_from_hrv_model.predict(x_hrv)
    probs_steps = s_from_step_model.predict(x_steps)
    probs_ox = s_from_ox_model.predict(x_ox)

    # Convert to class labels
    #pred_hrv = probs_hrv
    pred_steps = int(np.argmax(probs_steps))
    pred_ox = int(np.argmax(probs_ox))
    pred_hrv = int(predict_from_hrv_features(hrv_features))


    return {
        "Stress_from_HRV": pred_hrv,
        "Stress_from_Steps": pred_steps,
        "Stress_from_OxSleep": pred_ox,
        "Total Stress": pred_hrv + pred_steps + pred_ox
        }
'''

In [798]:
def generate_features_for_row(sub, seg, df, sampling_rate=250, noise_level=0.3, add_noise=True, strict=False):
    row = df[(df["sub"] == sub) & (df["seg"] == seg)]
    if row.empty:
        raise ValueError(f"No data found for sub={sub}, seg={seg}")

    raw_signal = row["raw data"].iloc[0]

    try:
        hrv_features = extract_hrv_features(raw_signal, sampling_rate=sampling_rate)
    except Exception as e:
        print(f"❌ HRV extraction error: {e}")
        hrv_features = [np.nan] * 34

    if np.any(np.isnan(hrv_features)) or np.any(np.isinf(hrv_features)):
        if strict:
            raise ValueError(f"❌ Invalid HRV features for sub={sub}, seg={seg}")
        else:
            hrv_features = [np.nan] * 34

    if add_noise and not np.any(np.isnan(hrv_features)):
        hrv_features = [
            val + np.random.normal(0, noise_level * max(abs(val), 1e-6))
            for val in hrv_features
        ]

    blood_oxygen = round(np.random.uniform(90.0, 99.0), 1)
    sleeping_hours = round(np.random.uniform(0.0, 9.0), 1)
    step_count = np.random.randint(0, 2000)

    return hrv_features[:34] + [blood_oxygen, sleeping_hours, step_count]


In [799]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model
model = s_from_hrv_model
scaler = scaler_hrv
encoder = encoder_hrv

def predict_from_hrv_features(hrv_features):
    try:
        features_df = pd.DataFrame([hrv_features], columns=X.columns.tolist())
        scaled = scaler.transform(features_df).reshape(1, 1, 34)
    except:
        scaled = scaler.transform([hrv_features]).reshape(1, 1, 34)

    prediction = model.predict(scaled)
    predicted_index = np.argmax(prediction)
    predicted_label = encoder.categories_[0][predicted_index]
    #return predicted_label, prediction,predicted_index
    return predicted_index

In [800]:
def predict_stress_from_all_models(sub, seg, df):
    try:
        # Step 1: Extract all features (HRV + physiology)
        full_features = generate_features_for_row(sub, seg, df,noise_level=0.3, add_noise=True, strict=False)
    except ValueError as e:
        print(f"⚠️ Feature extraction failed: {e}")
        return None

    if full_features is None or len(full_features) != 37:
        print(f"⚠️ Invalid feature vector for sub={sub}, seg={seg}")
        return None

    # Step 2: Split features
    hrv_features = full_features[:34]
    ox = full_features[34]
    sleep = full_features[35]
    steps = full_features[36]

    # Step 3: Scale steps and ox/sleep using pre-fitted scalers
    x_steps = scaler_steps.transform([[steps]])       # shape (1, 1)
    x_oxsleep = scaler_oxsleep.transform([[ox, sleep]])  # shape (1, 2)

    # Step 4: Predict from models
    pred_hrv = int(predict_from_hrv_features(hrv_features))  # returns class index
    pred_steps = int(np.argmax(s_from_step_model.predict(x_steps)))
    pred_ox = int(np.argmax(s_from_ox_model.predict(x_oxsleep)))

    # Step 5: Return all predictions
    return {
        "Stress_from_HRV": pred_hrv,
        "Stress_from_Steps": pred_steps,
        "Stress_from_OxSleep": pred_ox,
        "Total Stress": pred_hrv + pred_steps + pred_ox
    }

In [801]:
result = predict_stress_from_all_models(sub=10, seg=3, df=df)
print(result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
{'Stress_from_HRV': 1, 'Stress_from_Steps': 2, 'Stress_from_OxSleep': 2, 'Total Stress': 5}


In [803]:
'''def get_test_features(index, df=test_df):
    row = df.iloc[index]
    features = row.drop(labels=["datasetId", "condition"]).values.tolist()
    return features
real_row = get_test_features(6)
synthetic_row = [val + np.random.normal(0, 0.1 * abs(val)) for val in real_row]
pred = predict_from_hrv_features(synthetic_row)
#print("Predicted label:", label)
#print("Probabilities:", probs)
print("Predicted index:", idx)'''

'def get_test_features(index, df=test_df):\n    row = df.iloc[index]\n    features = row.drop(labels=["datasetId", "condition"]).values.tolist()\n    return features\nreal_row = get_test_features(6)\nsynthetic_row = [val + np.random.normal(0, 0.1 * abs(val)) for val in real_row]\npred = predict_from_hrv_features(synthetic_row)\n#print("Predicted label:", label)\n#print("Probabilities:", probs)\nprint("Predicted index:", idx)'

In [802]:
#features = generate_features_for_row(sub=10, seg=3, df=df, add_noise=True)
#label, probs = predict_from_hrv_features(features)
#print("Predicted label:", label)
#print("Probabilities:", probs)

In [804]:
# Create empty columns to store predictions
df["Stress_from_HRV"] = np.nan
df["Stress_from_Steps"] = np.nan
df["Stress_from_OxSleep"] = np.nan
df["Total_Stress"] = np.nan

# Loop over all rows
for idx, row in df.iterrows():
    sub = row["sub"]
    seg = row["seg"]

    stress_predictions = predict_stress_from_all_models(sub, seg, df)

    if predictions:
        df.at[idx, "Stress_from_HRV"] = stress_predictions["Stress_from_HRV"]
        df.at[idx, "Stress_from_Steps"] = stress_predictions["Stress_from_Steps"]
        df.at[idx, "Stress_from_OxSleep"] = stress_predictions["Stress_from_OxSleep"]
        df.at[idx, "Total Stress"] = stress_predictions["Total Stress"]
    else:
        print(f"⚠️ Skipped prediction for sub={sub}, seg={seg}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 131ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 96ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 127ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 210ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 175ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 144ms/step
1/1 ━━

In [805]:
df.groupby('Stress_from_HRV').size()

,0
Stress_from_HRV,
0.0,20
1.0,64
2.0,15


In [806]:
df.groupby('Stress_from_Steps').size()

,0
Stress_from_Steps,
0.0,5
1.0,4
2.0,90


In [807]:
df.groupby('Stress_from_OxSleep').size()

,0
Stress_from_OxSleep,
0.0,2
1.0,49
2.0,23
3.0,22
4.0,3


In [808]:
df.groupby('Total Stress').size()

,0
Total Stress,
2.0,2
3.0,10
4.0,41
5.0,24
6.0,21
7.0,1


# Emotion service

In [826]:
predoctions = predict_stress_from_all_models(sub=10, seg=3, df=df)
print(predoctions)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 63ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
{'Stress_from_HRV': 1, 'Stress_from_Steps': 2, 'Stress_from_OxSleep': 3, 'Total Stress': 6}


In [863]:
import neurokit2 as nk
from scipy.signal import find_peaks
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.stats import zscore
from scipy.integrate import trapezoid
from scipy.interpolate import interp1d
from scipy.signal import welch
from tensorflow import keras
from pyhrv.frequency_domain import welch_psd
from pyhrv.nonlinear import sample_entropy
import os
import keras
import joblib
import ast
########################################################################
from google.colab import drive
drive.mount('/content/drive')
########################################################################
project_dir = '/content/drive/MyDrive/Colab Notebooks/project/'
trained_models_dir = os.path.join(project_dir, 'trained models/')

# Check if the file exists before attempting to load
hrv_model_path = trained_models_dir+"stress models/keras/stress_from_hrv_model_up.keras"

s_from_hrv_model = keras.models.load_model(trained_models_dir+"stress models/keras/stress_model_up.keras")
s_from_step_model = keras.models.load_model(trained_models_dir+"stress models/keras/stress_from_steps_model.keras")
s_from_ox_model = keras.models.load_model(trained_models_dir+"stress models/keras/stress_from_sleep_and_ox_model.keras")

scaler_steps = joblib.load(trained_models_dir+"stress models/pkl/steps_scaler.pkl")
scaler_oxsleep = joblib.load(trained_models_dir+"stress models/pkl/ox_and_sleep_scaler.pkl")
scaler_hrv = joblib.load(trained_models_dir+'stress models/pkl/hrv_scaler.pkl')

encoder_hrv = joblib.load(trained_models_dir+'stress models/hrv_encoder.pkl')
########################################################################
save_dir = project_dir + 'Integration and simulation/'
original_df = pd.read_csv(save_dir+'ecg_segments_loaded.csv')
df = original_df.copy()
df["raw data"] = df["raw data"].apply(ast.literal_eval)
########################################################################
def extract_hrv_features(ecg_signal, sampling_rate=250):#, datasetId=None):
    # 1. R-peak detection using NeuroKit2
    _, rpeaks_info = nk.ecg_peaks(ecg_signal, sampling_rate=sampling_rate)
    rpeaks = rpeaks_info["ECG_R_Peaks"]  # indices of R-peaks in the signal

    # 2. Compute RR intervals (ms) from consecutive R-peak indices
    rr_intervals = np.diff(rpeaks) / sampling_rate * 1000.0  # convert index difference to milliseconds

    # 3. Remove RR outliers (<300 ms or >2000 ms) to clean artifacts
    rr_intervals = rr_intervals[(rr_intervals >= 300) & (rr_intervals <= 2000)]

    # If not enough intervals remain, you may handle separately (not covered here for brevity)

    # 4. Time-domain features
    # Basic statistics on RR intervals
    mean_rr   = float(np.mean(rr_intervals))
    median_rr = float(np.median(rr_intervals))
    sdrr      = float(np.std(rr_intervals, ddof=0))            # SDNN: std of RR
    # Successive differences of RR intervals
    diff_rr   = np.diff(rr_intervals)
    if diff_rr.size > 0:
        rmssd = float(np.sqrt(np.mean(diff_rr**2)))            # RMSSD
        sdsd  = float(np.std(diff_rr, ddof=0))                 # SDSD
    else:
        rmssd = 0.0
        sdsd  = 0.0
    sdrr_rmssd = float(sdrr / rmssd) if rmssd != 0 else 0.0    # ratio SDNN/RMSSD
    # Heart rate (beats per minute) from mean RR (ms)
    hr = float(0.0 if mean_rr == 0 else (60000.0 / mean_rr))
    # pNN25 and pNN50 (percentage of absolute differences above 25 ms and 50 ms)
    abs_diff_rr = np.abs(diff_rr)
    pnn25 = float(100.0 * np.sum(abs_diff_rr > 25) / abs_diff_rr.size) if abs_diff_rr.size > 0 else 0.0
    pnn50 = float(100.0 * np.sum(abs_diff_rr > 50) / abs_diff_rr.size) if abs_diff_rr.size > 0 else 0.0

    # 5. Poincaré plot features (SD1, SD2) and distribution shape (kurtosis, skewness)
    sd1 = float(sdsd / np.sqrt(2.0))
    sd2 = float(np.sqrt(2.0 * (sdrr**2) - 0.5 * (sdsd**2))) if sdrr >= 0 and sdsd >= 0 else 0.0
    # Kurtosis and skewness of RR intervals
    if rr_intervals.size > 0:
        # Using scipy.stats if available; otherwise, compute manually
        # Here we compute population statistics (Fisher's definition for kurtosis)
        m_rr = mean_rr
        std_rr = np.std(rr_intervals, ddof=0)
        if std_rr == 0:
            kurt = 0.0
            skew = 0.0
        else:
            z_rr = (rr_intervals - m_rr) / std_rr
            kurt = float(np.mean(z_rr**4) - 3)  # excess kurtosis
            skew = float(np.mean(z_rr**3))
    else:
        kurt = 0.0
        skew = 0.0

    # 6. Relative RR interval features (ratio of successive RR intervals)
    if rr_intervals.size > 1:
        rel_rr = rr_intervals[1:] / rr_intervals[:-1]  # ratio of each interval to the previous
    else:
        rel_rr = np.array([])
    if rel_rr.size > 0:
        mean_rel_rr   = float(np.mean(rel_rr))
        median_rel_rr = float(np.median(rel_rr))
        sdrr_rel_rr   = float(np.std(rel_rr, ddof=0))
    else:
        mean_rel_rr = 0.0
        median_rel_rr = 0.0
        sdrr_rel_rr = 0.0
    # Differences of relative RR series
    diff_rel_rr = np.diff(rel_rr)
    if diff_rel_rr.size > 0:
        rmssd_rel_rr = float(np.sqrt(np.mean(diff_rel_rr**2)))
        sdsd_rel_rr  = float(np.std(diff_rel_rr, ddof=0))
    else:
        rmssd_rel_rr = 0.0
        sdsd_rel_rr  = 0.0
    sdrr_rmssd_rel_rr = float(sdrr_rel_rr / rmssd_rel_rr) if rmssd_rel_rr != 0 else 0.0
    # Kurtosis and skewness of relative RR series
    if rel_rr.size > 0:
        m_rel = mean_rel_rr
        std_rel = np.std(rel_rr, ddof=0)
        if std_rel == 0:
            kurt_rel_rr = 0.0
            skew_rel_rr = 0.0
        else:
            z_rel = (rel_rr - m_rel) / std_rel
            kurt_rel_rr = float(np.mean(z_rel**4) - 3)
            skew_rel_rr = float(np.mean(z_rel**3))
    else:
        kurt_rel_rr = 0.0
        skew_rel_rr = 0.0

    # 7. Frequency-domain features using Welch's PSD (pyHRV)
    # Compute PSD and frequency-domain metrics
    fd_res = welch_psd(nni=rr_intervals, show=False, show_param=False)
    # Absolute powers [ms^2] in VLF, LF, HF bands
    vlf, lf, hf = fd_res['fft_abs']
    # Relative powers [% of total power]
    vlf_pct, lf_pct, hf_pct = fd_res['fft_rel']
    # Normalized units (LF, HF)
    lf_nu, hf_nu = fd_res['fft_norm']
    # Total power and LF/HF ratio
    tp = fd_res['fft_total']
    lf_hf = fd_res['fft_ratio']
    hf_lf = float(hf / lf) if lf != 0 else float('inf')

    # 8. Non-linear features: Sample Entropy and Higuchi Fractal Dimension
    sampen_result = sample_entropy(rr_intervals)
    sampen_val = sampen_result[0]
    #print(f"Extracted sampen_val from rr_intervals: {sampen_val}, Type: {type(sampen_val)}")

    # Higuchi fractal dimension using NeuroKit2
    #hfd_val, _ = nk.fractal_higuchi(rr_intervals, show=False)
    hfd_tuple = nk.fractal_higuchi(rr_intervals, show=False)
    hfd_val = hfd_tuple[0]
    #print(f"Extracted hfd_val from tuple: {hfd_tuple}, Type: {type(hfd_tuple)}")


    # 9. Compile all features into a list in the specified order
    features = [
        mean_rr, median_rr, sdrr, rmssd, sdsd, sdrr_rmssd, hr, pnn25, pnn50,
        sd1, sd2, kurt, skew,
        mean_rel_rr, median_rel_rr, sdrr_rel_rr, rmssd_rel_rr, sdsd_rel_rr, sdrr_rmssd_rel_rr, kurt_rel_rr, skew_rel_rr,
        float(vlf), float(vlf_pct), float(lf), float(lf_pct), float(lf_nu), float(hf), float(hf_pct), float(hf_nu),
        float(tp), float(lf_hf), float(hf_lf),
        float(sampen_val), float(hfd_val)#,
        #datasetId
    ]
    return features
########################################################################
def generate_features_for_row(sub, seg, df, sampling_rate=250, noise_level=0.3, add_noise=True, strict=False):
    row = df[(df["sub"] == sub) & (df["seg"] == seg)]
    if row.empty:
        raise ValueError(f"No data found for sub={sub}, seg={seg}")

    raw_signal = row["raw data"].iloc[0]

    try:
        hrv_features = extract_hrv_features(raw_signal, sampling_rate=sampling_rate)
    except Exception as e:
        print(f"❌ HRV extraction error: {e}")
        hrv_features = [np.nan] * 34

    if np.any(np.isnan(hrv_features)) or np.any(np.isinf(hrv_features)):
        if strict:
            raise ValueError(f"❌ Invalid HRV features for sub={sub}, seg={seg}")
        else:
            hrv_features = [np.nan] * 34

    if add_noise and not np.any(np.isnan(hrv_features)):
        hrv_features = [
            val + np.random.normal(0, noise_level * max(abs(val), 1e-6))
            for val in hrv_features
        ]

    blood_oxygen = round(np.random.uniform(90.0, 99.0), 1)
    sleeping_hours = round(np.random.uniform(0.0, 9.0), 1)
    step_count = np.random.randint(0, 2000)

    return hrv_features[:34] + [blood_oxygen, sleeping_hours, step_count]
########################################################################
def predict_from_hrv_features(hrv_features):
    """
    Predict stress level from HRV features using a pre-trained model.
    Handles feature name warnings properly by suppressing them.

    Args:
        hrv_features (list): List of 34 HRV features

    Returns:
        int: Predicted stress level index
    """
    try:
        # Suppress the specific warning we're seeing
        import warnings
        from sklearn.exceptions import DataConversionWarning
        warnings.filterwarnings('ignore', category=UserWarning, module='sklearn.utils.validation')

        # Create a properly formatted array
        hrv_array = np.array(hrv_features).reshape(1, -1)

        # Scale the features directly - we're suppressing the warnings
        scaled_features = scaler_hrv.transform(hrv_array)

        # Reshape for the model (add time dimension for LSTM)
        scaled = scaled_features.reshape(1, 1, 34)

        # Make prediction with verbose=0 to avoid progress messages
        prediction = s_from_hrv_model.predict(scaled, verbose=0)

        # Get predicted class and convert to native Python int
        predicted_index = int(np.argmax(prediction))

        return predicted_index

    except Exception as e:
        print(f"Error in HRV prediction: {e}")
        return 0  # Default to 0 on error

########################################################################
def predict_stress_from_all_models(sub, seg, df, add_noise=True):
    try:
        # Step 1: Extract all features (HRV + physiology)
        full_features = generate_features_for_row(sub, seg, df, noise_level=0.3, add_noise=add_noise, strict=False)
    except ValueError as e:
        print(f"⚠️ Feature extraction failed: {e}")
        return None

    if full_features is None or len(full_features) != 37:
        print(f"⚠️ Invalid feature vector for sub={sub}, seg={seg}")
        return None

    # Step 2: Split features
    hrv_features = full_features[:34]
    ox = full_features[34]
    sleep = full_features[35]
    steps = full_features[36]

    # Step 3: Scale steps and ox/sleep using pre-fitted scalers
    x_steps = scaler_steps.transform([[steps]])       # shape (1, 1)
    x_oxsleep = scaler_oxsleep.transform([[ox, sleep]])  # shape (1, 2)

    # Step 4: Predict from models
    pred_hrv = int(predict_from_hrv_features(hrv_features))  # returns class index
    pred_steps = int(np.argmax(s_from_step_model.predict(x_steps)))
    pred_ox = int(np.argmax(s_from_ox_model.predict(x_oxsleep)))

    # Step 5: Return all predictions
    return {
        "Stress_from_HRV": pred_hrv,
        "Stress_from_Steps": pred_steps,
        "Stress_from_OxSleep": pred_ox,
        "Total Stress": pred_hrv + pred_steps + pred_ox
    }
########################################################################


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [864]:
def test_prediction_with_and_without_noise():
    """
    Test the prediction function with and without noise to verify stability.
    """
    # Choose a sample row
    sample_sub = df["sub"].iloc[0]
    sample_seg = df["seg"].iloc[0]

    print(f"Testing prediction for sub={sample_sub}, seg={sample_seg}")

    # Generate features without noise
    print("\n--- Without Noise ---")
    features_no_noise = generate_features_for_row(sample_sub, sample_seg, df, add_noise=False)
    hrv_features_no_noise = features_no_noise[:34]

    # Make prediction without noise
    pred_no_noise = predict_from_hrv_features(hrv_features_no_noise)
    print(f"Prediction without noise: {pred_no_noise}")

    # Generate features with noise
    print("\n--- With Noise ---")
    features_with_noise = generate_features_for_row(sample_sub, sample_seg, df, add_noise=True)
    hrv_features_with_noise = features_with_noise[:34]

    # Make prediction with noise
    pred_with_noise = predict_from_hrv_features(hrv_features_with_noise)
    print(f"Prediction with noise: {pred_with_noise}")

    # Check distribution of predictions for multiple runs with noise
    print("\n--- Multiple Runs With Noise ---")
    noise_predictions = []
    for i in range(10):
        features = generate_features_for_row(sample_sub, sample_seg, df, add_noise=True)
        hrv_features = features[:34]
        pred = predict_from_hrv_features(hrv_features)
        noise_predictions.append(pred)

    print(f"Predictions with noise (10 runs): {noise_predictions}")
    print(f"Unique predictions: {set(noise_predictions)}")
    print(f"Distribution: {pd.Series(noise_predictions).value_counts()}")

    # Check distribution for multiple rows without noise
    print("\n--- Multiple Samples Without Noise ---")
    sample_rows = min(10, len(df))
    no_noise_predictions = []

    for i in range(sample_rows):
        sub = df["sub"].iloc[i]
        seg = df["seg"].iloc[i]
        try:
            features = generate_features_for_row(sub, seg, df, add_noise=False)
            hrv_features = features[:34]
            pred = predict_from_hrv_features(hrv_features)
            no_noise_predictions.append(pred)
            print(f"Sample {i+1}: sub={sub}, seg={seg}, prediction={pred}")
        except Exception as e:
            print(f"Error processing sample {i+1}: {e}")

    print(f"\nPredictions without noise ({len(no_noise_predictions)} samples): {no_noise_predictions}")
    print(f"Unique predictions: {set(no_noise_predictions)}")
    print(f"Distribution: {pd.Series(no_noise_predictions).value_counts()}")


# Run the test
test_prediction_with_and_without_noise()

Testing prediction for sub=10, seg=1

--- Without Noise ---
Prediction without noise: 1

--- With Noise ---
Prediction with noise: 1

--- Multiple Runs With Noise ---
Predictions with noise (10 runs): [1, 1, 0, 0, 2, 2, 1, 1, 1, 1]
Unique predictions: {0, 1, 2}
Distribution: 1    6
0    2
2    2
Name: count, dtype: int64

--- Multiple Samples Without Noise ---
Sample 1: sub=10, seg=1, prediction=1
Sample 2: sub=10, seg=2, prediction=1
Sample 3: sub=10, seg=3, prediction=1
Sample 4: sub=10, seg=4, prediction=1
Sample 5: sub=10, seg=5, prediction=1
Sample 6: sub=11, seg=1, prediction=1
Sample 7: sub=11, seg=2, prediction=1
Sample 8: sub=11, seg=3, prediction=1
Sample 9: sub=11, seg=4, prediction=1
Sample 10: sub=11, seg=5, prediction=1

Predictions without noise (10 samples): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Unique predictions: {1}
Distribution: 1    10
Name: count, dtype: int64


In [ ]:
import os
import numpy as np
import pandas as pd
import random
import joblib
import keras
import neurokit2 as nk
from pyhrv.frequency_domain import welch_psd
from nolds import sampen as sample_entropy

class EmotionService:
    def __init__(self):
        project_dir = '/content/drive/MyDrive/Colab Notebooks/project/'
        trained_models_dir = os.path.join(project_dir, 'trained models/')

        # Load models if they exist
        hrv_model_path = os.path.join(trained_models_dir, "stress models/keras/stress_from_hrv_model.keras")
        if not os.path.exists(hrv_model_path):
            raise FileNotFoundError(f"Model file not found at {hrv_model_path}")

        self.s_from_hrv_model = keras.models.load_model(hrv_model_path)
        self.s_from_step_model = keras.models.load_model(
            os.path.join(trained_models_dir, "stress models/keras/stress_from_steps_model.keras"))
        self.s_from_ox_model = keras.models.load_model(
            os.path.join(trained_models_dir, "stress models/keras/stress_from_sleep_and_ox_model.keras"))

        self.scaler_steps = joblib.load(
            os.path.join(trained_models_dir, "stress models/pkl/steps_scaler.pkl"))
        self.scaler_oxsleep = joblib.load(
            os.path.join(trained_models_dir, "stress models/pkl/ox_and_sleep_scaler.pkl"))

        print("✅ EmotionService initialized with models and scalers")

    def extract_hrv_features(self, ecg_signal, sampling_rate=250):
        _, rpeaks_info = nk.ecg_peaks(ecg_signal, sampling_rate=sampling_rate)
        rpeaks = rpeaks_info["ECG_R_Peaks"]

        rr_intervals = np.diff(rpeaks) / sampling_rate * 1000.0
        rr_intervals = rr_intervals[(rr_intervals >= 300) & (rr_intervals <= 2000)]

        mean_rr = float(np.mean(rr_intervals))
        median_rr = float(np.median(rr_intervals))
        sdrr = float(np.std(rr_intervals, ddof=0))
        diff_rr = np.diff(rr_intervals)
        rmssd = float(np.sqrt(np.mean(diff_rr**2))) if diff_rr.size > 0 else 0.0
        sdsd = float(np.std(diff_rr, ddof=0)) if diff_rr.size > 0 else 0.0
        sdrr_rmssd = float(sdrr / rmssd) if rmssd != 0 else 0.0
        hr = float(0.0 if mean_rr == 0 else (60000.0 / mean_rr))
        abs_diff_rr = np.abs(diff_rr)
        pnn25 = float(100.0 * np.sum(abs_diff_rr > 25) / abs_diff_rr.size) if abs_diff_rr.size > 0 else 0.0
        pnn50 = float(100.0 * np.sum(abs_diff_rr > 50) / abs_diff_rr.size) if abs_diff_rr.size > 0 else 0.0
        sd1 = float(sdsd / np.sqrt(2.0))
        sd2 = float(np.sqrt(2.0 * (sdrr**2) - 0.5 * (sdsd**2))) if sdrr >= 0 and sdsd >= 0 else 0.0

        std_rr = np.std(rr_intervals, ddof=0)
        if std_rr == 0:
            kurt = 0.0
            skew = 0.0
        else:
            z_rr = (rr_intervals - mean_rr) / std_rr
            kurt = float(np.mean(z_rr**4) - 3)
            skew = float(np.mean(z_rr**3))

        rel_rr = rr_intervals[1:] / rr_intervals[:-1] if rr_intervals.size > 1 else np.array([])
        mean_rel_rr = float(np.mean(rel_rr)) if rel_rr.size > 0 else 0.0
        median_rel_rr = float(np.median(rel_rr)) if rel_rr.size > 0 else 0.0
        sdrr_rel_rr = float(np.std(rel_rr, ddof=0)) if rel_rr.size > 0 else 0.0
        diff_rel_rr = np.diff(rel_rr)
        rmssd_rel_rr = float(np.sqrt(np.mean(diff_rel_rr**2))) if diff_rel_rr.size > 0 else 0.0
        sdsd_rel_rr = float(np.std(diff_rel_rr, ddof=0)) if diff_rel_rr.size > 0 else 0.0
        sdrr_rmssd_rel_rr = float(sdrr_rel_rr / rmssd_rel_rr) if rmssd_rel_rr != 0 else 0.0

        std_rel = np.std(rel_rr, ddof=0)
        if std_rel == 0:
            kurt_rel_rr = 0.0
            skew_rel_rr = 0.0
        else:
            z_rel = (rel_rr - mean_rel_rr) / std_rel
            kurt_rel_rr = float(np.mean(z_rel**4) - 3)
            skew_rel_rr = float(np.mean(z_rel**3))

        fd_res = welch_psd(nni=rr_intervals, show=False, show_param=False)
        vlf, lf, hf = fd_res['fft_abs']
        vlf_pct, lf_pct, hf_pct = fd_res['fft_rel']
        lf_nu, hf_nu = fd_res['fft_norm']
        tp = fd_res['fft_total']
        lf_hf = fd_res['fft_ratio']
        hf_lf = float(hf / lf) if lf != 0 else float('inf')

        sampen_val = sample_entropy(rr_intervals)
        hfd_val, _ = nk.fractal_higuchi(rr_intervals, show=False)

        return [
            mean_rr, median_rr, sdrr, rmssd, sdsd, sdrr_rmssd, hr, pnn25, pnn50,
            sd1, sd2, kurt, skew,
            mean_rel_rr, median_rel_rr, sdrr_rel_rr, rmssd_rel_rr, sdsd_rel_rr,
            sdrr_rmssd_rel_rr, kurt_rel_rr, skew_rel_rr,
            float(vlf), float(vlf_pct), float(lf), float(lf_pct), float(lf_nu),
            float(hf), float(hf_pct), float(hf_nu), float(tp), float(lf_hf),
            float(hf_lf), float(sampen_val), float(hfd_val), datasetId
        ]

    def generate_features_for_row(self, sub, seg, df, sampling_rate=250):
        row = df[(df["sub"] == sub) & (df["seg"] == seg)]
        if row.empty:
            raise ValueError(f"No data found for sub={sub}, seg={seg}")

        raw_signal = row["raw data"].iloc[0]

        try:
            hrv_features = self.extract_hrv_features(raw_signal, sampling_rate=sampling_rate, datasetId=sub)
        except Exception as e:
            print(f"❌ Failed to extract HRV features for sub={sub}, seg={seg}: {e}")
            hrv_features = [np.nan] * 34

        blood_oxygen = round(np.random.uniform(75.0, 99.0), 1)
        sleeping_hours = round(np.random.uniform(0.0, 9.0), 1)
        step_count = np.random.randint(0, 201)

        return hrv_features[:34] + [blood_oxygen, sleeping_hours, step_count]

    def predict_stress_from_all_models(self, sub, seg, df):
        features = self.generate_features_for_row(sub, seg, df)
        if features is None or len(features) != 37:
            print(f"⚠️ Feature extraction failed for sub={sub}, seg={seg}")
            return None

        hrv_features = np.array(features[:34], dtype=np.float32).reshape(1, 1, 34)
        ox = features[34]
        sleep = features[35]
        steps = features[36]

        x_steps = self.scaler_steps.transform([[steps]])
        x_ox = self.scaler_oxsleep.transform([[ox, sleep]])

        pred_hrv = int(np.argmax(self.s_from_hrv_model.predict(hrv_features)))
        pred_steps = int(np.argmax(self.s_from_step_model.predict(x_steps)))
        pred_ox = int(np.argmax(self.s_from_ox_model.predict(x_ox)))

        return {
            "Stress_from_HRV": pred_hrv,
            "Stress_from_Steps": pred_steps,
            "Stress_from_OxSleep": pred_ox,
            "Total Stress": pred_hrv + pred_steps + pred_ox
        }

    def detect(self, df):
        pairs = [(row["sub"], row["seg"]) for _, row in df.iterrows()]
        sub, seg = random.choice(pairs)
        print(f"Detect emotion from row: sub={sub}, seg={seg}")

        stress_predictions = self.predict_stress_from_all_models(sub, seg, df)

        stress_value = stress_predictions["Total Stress"]
        if stress_value >= 6:
            emotion = "anger"
        elif stress_value >= 4:
            emotion = "happy"
        elif stress_value >= 2:
            emotion = "sad"
        else:
            emotion = "relaxed"

        print(f"✅ Detected emotion: {emotion}")
        return emotion


In [850]:
import os
import numpy as np
import pandas as pd
import random
import joblib
import keras
import neurokit2 as nk
from pyhrv.frequency_domain import welch_psd
from pyhrv.nonlinear import sample_entropy


class EmotionService:
    def __init__(self):
        project_dir = '/content/drive/MyDrive/Colab Notebooks/project/'
        trained_models_dir = os.path.join(project_dir, 'trained models/')

        # Load models
        self.s_from_hrv_model = keras.models.load_model(
            os.path.join(trained_models_dir, "stress models/keras/stress_model_up.keras"))
        self.s_from_step_model = keras.models.load_model(
            os.path.join(trained_models_dir, "stress models/keras/stress_from_steps_model.keras"))
        self.s_from_ox_model = keras.models.load_model(
            os.path.join(trained_models_dir, "stress models/keras/stress_from_sleep_and_ox_model.keras"))

        # Load scalers and encoder
        self.scaler_steps = joblib.load(
            os.path.join(trained_models_dir, "stress models/pkl/steps_scaler.pkl"))
        self.scaler_oxsleep = joblib.load(
            os.path.join(trained_models_dir, "stress models/pkl/ox_and_sleep_scaler.pkl"))
        self.scaler_hrv = joblib.load(
            os.path.join(trained_models_dir, "stress models/pkl/hrv_scaler.pkl"))
        self.encoder_hrv = joblib.load(
            os.path.join(trained_models_dir, "stress models/hrv_encoder.pkl"))

        print("✅ EmotionService initialized with models and scalers")

    def extract_hrv_features(self, ecg_signal, sampling_rate=250):
        try:
            _, rpeaks_info = nk.ecg_peaks(ecg_signal, sampling_rate=sampling_rate)
            rpeaks = rpeaks_info["ECG_R_Peaks"]
            rr_intervals = np.diff(rpeaks) / sampling_rate * 1000.0
            rr_intervals = rr_intervals[(rr_intervals >= 300) & (rr_intervals <= 2000)]

            if len(rr_intervals) < 2:
                raise ValueError("Not enough RR intervals")

            mean_rr = float(np.mean(rr_intervals))
            median_rr = float(np.median(rr_intervals))
            sdrr = float(np.std(rr_intervals))
            diff_rr = np.diff(rr_intervals)
            rmssd = float(np.sqrt(np.mean(diff_rr**2))) if len(diff_rr) > 0 else 0.0
            sdsd = float(np.std(diff_rr)) if len(diff_rr) > 0 else 0.0
            sdrr_rmssd = float(sdrr / rmssd) if rmssd != 0 else 0.0
            hr = float(60000.0 / mean_rr) if mean_rr != 0 else 0.0
            abs_diff_rr = np.abs(diff_rr)
            pnn25 = float(100.0 * np.sum(abs_diff_rr > 25) / len(abs_diff_rr)) if len(abs_diff_rr) > 0 else 0.0
            pnn50 = float(100.0 * np.sum(abs_diff_rr > 50) / len(abs_diff_rr)) if len(abs_diff_rr) > 0 else 0.0

            sd1 = float(sdsd / np.sqrt(2.0))
            sd2 = float(np.sqrt(2.0 * (sdrr**2) - 0.5 * (sdsd**2))) if sdrr >= 0 and sdsd >= 0 else 0.0

            std_rr = np.std(rr_intervals)
            kurt = float(np.mean(((rr_intervals - mean_rr) / std_rr)**4) - 3) if std_rr != 0 else 0.0
            skew = float(np.mean(((rr_intervals - mean_rr) / std_rr)**3)) if std_rr != 0 else 0.0

            rel_rr = rr_intervals[1:] / rr_intervals[:-1]
            mean_rel_rr = float(np.mean(rel_rr)) if len(rel_rr) > 0 else 0.0
            median_rel_rr = float(np.median(rel_rr)) if len(rel_rr) > 0 else 0.0
            sdrr_rel_rr = float(np.std(rel_rr)) if len(rel_rr) > 0 else 0.0
            diff_rel_rr = np.diff(rel_rr)
            rmssd_rel_rr = float(np.sqrt(np.mean(diff_rel_rr**2))) if len(diff_rel_rr) > 0 else 0.0
            sdsd_rel_rr = float(np.std(diff_rel_rr)) if len(diff_rel_rr) > 0 else 0.0
            sdrr_rmssd_rel_rr = float(sdrr_rel_rr / rmssd_rel_rr) if rmssd_rel_rr != 0 else 0.0

            std_rel = np.std(rel_rr)
            kurt_rel_rr = float(np.mean(((rel_rr - mean_rel_rr) / std_rel)**4) - 3) if std_rel != 0 else 0.0
            skew_rel_rr = float(np.mean(((rel_rr - mean_rel_rr) / std_rel)**3)) if std_rel != 0 else 0.0

            fd = welch_psd(nni=rr_intervals, show=False, show_param=False)
            vlf, lf, hf = fd["fft_abs"]
            vlf_pct, lf_pct, hf_pct = fd["fft_rel"]
            lf_nu, hf_nu = fd["fft_norm"]
            tp = fd["fft_total"]
            lf_hf = fd["fft_ratio"]
            hf_lf = float(hf / lf) if lf != 0 else float("inf")

            sampen_val = float(sample_entropy(rr_intervals)[0])
            hfd_val = float(nk.fractal_higuchi(rr_intervals, show=False)[0])

            return [
                mean_rr, median_rr, sdrr, rmssd, sdsd, sdrr_rmssd, hr, pnn25, pnn50,
                sd1, sd2, kurt, skew,
                mean_rel_rr, median_rel_rr, sdrr_rel_rr, rmssd_rel_rr, sdsd_rel_rr,
                sdrr_rmssd_rel_rr, kurt_rel_rr, skew_rel_rr,
                float(vlf), float(vlf_pct), float(lf), float(lf_pct), float(lf_nu),
                float(hf), float(hf_pct), float(hf_nu), float(tp), float(lf_hf),
                float(hf_lf), sampen_val, hfd_val
            ]
        except Exception as e:
            print(f"❌ HRV feature extraction failed: {e}")
            return [np.nan] * 34

    def generate_features_for_row(self, sub, seg, df, sampling_rate=250, noise_level=0.3, add_noise=True):
        row = df[(df["sub"] == sub) & (df["seg"] == seg)]
        if row.empty:
            raise ValueError(f"No data found for sub={sub}, seg={seg}")

        raw_signal = row["raw data"].iloc[0]
        hrv_features = self.extract_hrv_features(raw_signal, sampling_rate)

        if add_noise and not np.any(np.isnan(hrv_features)):
            hrv_features = [
                val + np.random.normal(0, noise_level * max(abs(val), 1e-6))
                for val in hrv_features
            ]

        blood_oxygen = round(np.random.uniform(90.0, 99.0), 1)
        sleeping_hours = round(np.random.uniform(0.0, 9.0), 1)
        step_count = np.random.randint(0, 2000)

        return hrv_features + [blood_oxygen, sleeping_hours, step_count]

    def predict_stress_from_all_models(self, sub, seg, df):
        try:
            features = self.generate_features_for_row(sub, seg, df)
        except Exception as e:
            print(f"❌ Feature generation failed: {e}")
            return None

        if features is None or len(features) != 37:
            print("⚠️ Feature vector invalid length")
            return None

        hrv_features = features[:34]
        ox = features[34]
        sleep = features[35]
        steps = features[36]

        x_hrv = self.scaler_hrv.transform(pd.DataFrame([hrv_features]))
        x_hrv = x_hrv.reshape(1, 1, 34)
        x_steps = self.scaler_steps.transform(pd.DataFrame([[steps]], columns=["steps"]))
        x_oxsleep = self.scaler_oxsleep.transform(pd.DataFrame([[ox, sleep]], columns=["blood_oxygen", "sleeping_hours"]))

        pred_hrv = int(np.argmax(self.s_from_hrv_model.predict(x_hrv)))
        pred_steps = int(np.argmax(self.s_from_step_model.predict(x_steps)))
        pred_ox = int(np.argmax(self.s_from_ox_model.predict(x_oxsleep)))

        return {
            "Stress_from_HRV": pred_hrv,
            "Stress_from_Steps": pred_steps,
            "Stress_from_OxSleep": pred_ox,
            "Total Stress": pred_hrv + pred_steps + pred_ox
        }

    def detect(self, df):
        pairs = [(row["sub"], row["seg"]) for _, row in df.iterrows()]
        sub, seg = random.choice(pairs)
        print(f"🔍 Detecting emotion for sub={sub}, seg={seg}")

        stress_predictions = self.predict_stress_from_all_models(sub, seg, df)

        if not stress_predictions:
            return "unknown"

        score = stress_predictions["Total Stress"]
        if score >= 6:
            emotion = "anger"
        elif score >= 4:
            emotion = "happy"
        elif score >= 2:
            emotion = "sad"
        else:
            emotion = "relaxed"

        print(f"✅ Detected emotion: {emotion}")
        return emotion


In [852]:
emotion_service = EmotionService()
detected_emotion = emotion_service.detect(df)

✅ EmotionService initialized with models and scalers
🔍 Detecting emotion for sub=10, seg=5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 116ms/step


/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/validation.py:2732: UserWarning: X has feature names, but StandardScaler was fitted without feature names
  warnings.warn(


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
✅ Detected emotion: happy


# Die already

In [876]:
def test_noise_sensitivity(sub, seg, df, noise_levels=[0, 0.05, 0.1, 0.15, 0.2, 0.25, 0.3]):
    """Test model predictions at different noise levels."""
    results = []

    # Run predictions with different noise levels
    for noise_level in noise_levels:
        predictions = []

        # Run multiple predictions at each noise level
        for _ in range(10):
            # Modify the noise level parameter in the function call
            full_features = generate_features_for_row(sub, seg, df, noise_level=noise_level, add_noise=True)
            hrv_features = full_features[:34]

            # Get prediction
            pred = predict_from_hrv_features(hrv_features)
            predictions.append(pred)

        # Record results
        results.append({
            'noise_level': noise_level,
            'predictions': predictions,
            'unique_preds': set(predictions),
            'most_common': max(set(predictions), key=predictions.count),
            'consistency': predictions.count(max(set(predictions), key=predictions.count)) / len(predictions)
        })

    # Display results
    for result in results:
        print(f"Noise level: {result['noise_level']:.2f}")
        print(f"Predictions: {result['predictions']}")
        print(f"Consistency: {result['consistency'] * 100:.0f}%")
        print()

    return results

In [877]:
results = test_noise_sensitivity(10, 1, df)

Noise level: 0.00
Predictions: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Consistency: 100%

Noise level: 0.05
Predictions: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Consistency: 100%

Noise level: 0.10
Predictions: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Consistency: 100%

Noise level: 0.15
Predictions: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Consistency: 100%

Noise level: 0.20
Predictions: [1, 1, 1, 0, 1, 2, 0, 0, 0, 1]
Consistency: 50%

Noise level: 0.25
Predictions: [1, 0, 0, 1, 1, 1, 1, 1, 1, 0]
Consistency: 70%

Noise level: 0.30
Predictions: [1, 1, 2, 1, 1, 0, 1, 1, 1, 0]
Consistency: 70%



In [889]:
def visualize_ecg_processing(sub, seg, df, sampling_rate=250):
    """Visualize the ECG processing steps for a specific segment."""
    # Get the ECG data
    row = df[(df["sub"] == sub) & (df["seg"] == seg)]
    if row.empty:
        raise ValueError(f"No data found for sub={sub}, seg={seg}")

    raw_signal = row["raw data"].iloc[0]

    # Convert raw_signal to a NumPy array
    raw_signal_np = np.array(raw_signal)

    # Process ECG for visualization
    # R-peak detection
    _, rpeaks_info = nk.ecg_peaks(raw_signal_np, sampling_rate=sampling_rate) # Use NumPy array here too
    rpeaks = rpeaks_info["ECG_R_Peaks"]

    # Calculate RR intervals
    rr_intervals = np.diff(rpeaks) / sampling_rate * 1000.0

    # Filter RR intervals
    valid_rr = rr_intervals[(rr_intervals >= 300) & (rr_intervals <= 2000)]
    filtered_out = rr_intervals[(rr_intervals < 300) | (rr_intervals > 2000)]

    # Set up figures
    fig, axes = plt.subplots(3, 1, figsize=(15, 12))

    # Plot raw ECG with R-peaks
    axes[0].plot(raw_signal_np) # Plot the NumPy array
    axes[0].scatter(rpeaks, raw_signal_np[rpeaks], color='red') # Use NumPy array for indexing
    axes[0].set_title(f'ECG Signal with {len(rpeaks)} R-peaks detected')

    # Plot RR intervals
    axes[1].plot(rr_intervals, '.-')
    axes[1].axhline(y=300, color='r', linestyle='--', label='Min threshold (300ms)')
    axes[1].axhline(y=2000, color='r', linestyle='--', label='Max threshold (2000ms)')
    axes[1].set_title(f'RR Intervals (removed: {len(filtered_out)}/{len(rr_intervals)})')
    axes[1].legend()

    # Plot histogram of valid RR intervals
    axes[2].hist(valid_rr, bins=30)
    axes[2].set_title(f'RR Interval Distribution (mean: {np.mean(valid_rr):.1f}ms)')

    plt.tight_layout()
    plt.show()

    # Print statistics
    print(f"R-peaks detected: {len(rpeaks)}")
    print(f"RR intervals: {len(rr_intervals)}")
    print(f"Valid RR intervals: {len(valid_rr)} ({len(valid_rr)/len(rr_intervals)*100:.1f}%)")

    return rpeaks, valid_rr

In [890]:
rpeaks, valid_rr = visualize_ecg_processing(10, 1, df)

R-peaks detected: 52
RR intervals: 51
Valid RR intervals: 51 (100.0%)


In [880]:
def analyze_feature_stability(sub, seg, df, num_runs=20):
    """Analyze how stable each HRV feature is when noise is added."""
    features_list = []
    predictions = []

    # Feature names for better visualization
    feature_names = [
        "mean_rr", "median_rr", "sdrr", "rmssd", "sdsd", "sdrr_rmssd", "hr", "pnn25", "pnn50",
        "sd1", "sd2", "kurt", "skew",
        "mean_rel_rr", "median_rel_rr", "sdrr_rel_rr", "rmssd_rel_rr", "sdsd_rel_rr",
        "sdrr_rmssd_rel_rr", "kurt_rel_rr", "skew_rel_rr",
        "vlf", "vlf_pct", "lf", "lf_pct", "lf_nu", "hf", "hf_pct", "hf_nu",
        "tp", "lf_hf", "hf_lf", "sampen_val", "hfd_val"
    ]

    # First, get features without noise for baseline
    baseline_features = generate_features_for_row(sub, seg, df, add_noise=False)
    baseline_hrv = baseline_features[:34]
    baseline_pred = predict_from_hrv_features(baseline_hrv)

    print(f"Baseline prediction (no noise): {baseline_pred}")

    # Run multiple times with noise
    for i in range(num_runs):
        features = generate_features_for_row(sub, seg, df, add_noise=True)
        hrv_features = features[:34]
        features_list.append(hrv_features)

        prediction = predict_from_hrv_features(hrv_features)
        predictions.append(prediction)

    # Convert to DataFrame for analysis
    features_df = pd.DataFrame(features_list, columns=feature_names)

    # Calculate coefficient of variation for each feature
    feature_means = features_df.mean()
    feature_stds = features_df.std()
    cv = (feature_stds / feature_means.abs()).fillna(0)

    # Sort features by their variability
    sorted_features = cv.sort_values(ascending=False)

    # Analyze prediction changes
    pred_counts = pd.Series(predictions).value_counts()
    pred_pcts = pred_counts / len(predictions) * 100

    print(f"Predictions with noise: {predictions}")
    print(f"Unique predictions: {set(predictions)}")
    print(f"Distribution:")
    for pred, count in pred_counts.items():
        print(f"  Class {pred}: {count} times ({pred_pcts[pred]:.1f}%)")

    print("\nTop 10 most variable features:")
    for idx in sorted_features.index[:10]:
        print(f"{idx}: CV={cv[idx]:.4f}, Mean={feature_means[idx]:.4f}, STD={feature_stds[idx]:.4f}")

    # Visualize feature changes that lead to different predictions
    unique_preds = sorted(set(predictions))
    if len(unique_preds) > 1:
        # Group features by prediction
        grouped_features = {}
        for i, pred in enumerate(predictions):
            if pred not in grouped_features:
                grouped_features[pred] = []
            grouped_features[pred].append(features_list[i])

        # Compare means of top varying features between prediction classes
        compare_df = pd.DataFrame()
        for pred, feats in grouped_features.items():
            pred_df = pd.DataFrame(feats, columns=feature_names)
            compare_df[f"Class_{pred}"] = pred_df.mean()

        # Plot differences for top features
        top_features = sorted_features.index[:5]
        plt.figure(figsize=(12, 8))
        compare_df.loc[top_features].plot(kind='bar')
        plt.title("Feature Means by Prediction Class")
        plt.ylabel("Feature Value")
        plt.tight_layout()
        plt.show()

    return features_df, sorted_features, compare_df if len(unique_preds) > 1 else None

In [881]:
features_df, sorted_features, compare_df = analyze_feature_stability(10, 1, df)

Baseline prediction (no noise): 1
Predictions with noise: [1, 1, 1, 2, 1, 0, 1, 2, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0]
Unique predictions: {0, 1, 2}
Distribution:
  Class 1: 14 times (70.0%)
  Class 0: 4 times (20.0%)
  Class 2: 2 times (10.0%)

Top 10 most variable features:
pnn25: CV=12.3908, Mean=-0.0000, STD=0.0000
pnn50: CV=2.0299, Mean=0.0000, STD=0.0000
median_rel_rr: CV=0.4017, Mean=0.8708, STD=0.3498
hf_nu: CV=0.3867, Mean=25.6452, STD=9.9169
sd1: CV=0.3758, Mean=2.5601, STD=0.9621
skew_rel_rr: CV=0.3755, Mean=0.0988, STD=0.0371
hfd_val: CV=0.3681, Mean=1.7900, STD=0.6590
sdrr_rmssd_rel_rr: CV=0.3641, Mean=0.5550, STD=0.2021
sdsd: CV=0.3525, Mean=3.5965, STD=1.2677
kurt_rel_rr: CV=0.3460, Mean=-0.6184, STD=0.2139


In [882]:
def check_signal_quality(sub, seg, df):
    """Analyze the ECG signal quality."""
    # Get the ECG data
    row = df[(df["sub"] == sub) & (df["seg"] == seg)]
    if row.empty:
        raise ValueError(f"No data found for sub={sub}, seg={seg}")

    raw_signal = row["raw data"].iloc[0]

    # Calculate signal quality metrics
    # 1. Mean and standard deviation
    signal_mean = np.mean(raw_signal)
    signal_std = np.std(raw_signal)

    # 2. Signal-to-noise ratio estimate (using median absolute deviation)
    signal_mad = np.median(np.abs(raw_signal - np.median(raw_signal)))
    snr_estimate = np.median(np.abs(raw_signal)) / (1.4826 * signal_mad) if signal_mad != 0 else float('inf')

    # 3. Check for flat lines or saturation
    diff_signal = np.diff(raw_signal)
    zero_runs = np.where(np.abs(diff_signal) < 1e-6)[0]
    flat_segments = len(zero_runs)
    flat_percentage = flat_segments / len(raw_signal) * 100

    # 4. Power spectral density
    freqs, psd = welch(raw_signal, fs=250)

    # Visualize
    fig, axes = plt.subplots(3, 1, figsize=(15, 12))

    # Plot raw signal
    axes[0].plot(raw_signal)
    axes[0].set_title(f'Raw ECG Signal (mean={signal_mean:.2f}, std={signal_std:.2f})')

    # Plot histogram
    axes[1].hist(raw_signal, bins=50)
    axes[1].set_title('Signal Amplitude Distribution')

    # Plot PSD
    axes[2].semilogy(freqs, psd)
    axes[2].set_xlabel('Frequency [Hz]')
    axes[2].set_ylabel('PSD [V^2/Hz]')
    axes[2].set_title('Power Spectral Density')
    axes[2].set_xlim(0, 40)  # Focus on physiological frequencies

    plt.tight_layout()
    plt.show()

    # Print statistics
    print(f"Signal statistics:")
    print(f"  Mean: {signal_mean:.4f}")
    print(f"  Std Dev: {signal_std:.4f}")
    print(f"  SNR estimate: {snr_estimate:.4f}")
    print(f"  Flat segments: {flat_percentage:.2f}% of signal")

    return {
        'mean': signal_mean,
        'std': signal_std,
        'snr': snr_estimate,
        'flat_pct': flat_percentage
    }

In [883]:
signal_stats = check_signal_quality(10, 1, df)

Signal statistics:
  Mean: 8.3442
  Std Dev: 58.9478
  SNR estimate: 0.7869
  Flat segments: 35.06% of signal


In [884]:
def analyze_model_confidence(sub, seg, df, num_runs=10):
    """Analyze the model's confidence in its predictions with/without noise."""
    # Get predictions without noise
    clean_features = generate_features_for_row(sub, seg, df, add_noise=False)
    clean_hrv = clean_features[:34]

    # Reshape for the model
    hrv_array = np.array(clean_hrv).reshape(1, -1)
    scaled_features = scaler_hrv.transform(hrv_array)
    scaled = scaled_features.reshape(1, 1, 34)

    # Get raw probabilities for clean prediction
    clean_probs = s_from_hrv_model.predict(scaled, verbose=0)[0]
    clean_pred = np.argmax(clean_probs)

    print("Without noise:")
    print(f"Prediction: Class {clean_pred}")
    print(f"Confidence: {clean_probs}")
    print(f"Highest confidence: {np.max(clean_probs):.4f}")

    # Run multiple predictions with noise
    print("\nWith noise:")
    noisy_results = []

    for i in range(num_runs):
        # Get noisy features
        noisy_features = generate_features_for_row(sub, seg, df, add_noise=True)
        noisy_hrv = noisy_features[:34]

        # Scale and predict
        hrv_array = np.array(noisy_hrv).reshape(1, -1)
        scaled_features = scaler_hrv.transform(hrv_array)
        scaled = scaled_features.reshape(1, 1, 34)

        # Get probabilities
        probs = s_from_hrv_model.predict(scaled, verbose=0)[0]
        pred = np.argmax(probs)
        confidence = np.max(probs)

        noisy_results.append({
            'prediction': pred,
            'confidence': confidence,
            'probabilities': probs
        })

        print(f"Run {i+1}: Class {pred}, confidence: {confidence:.4f}")

    # Analyze confidence distribution
    confidences = [r['confidence'] for r in noisy_results]
    predictions = [r['prediction'] for r in noisy_results]

    print(f"\nMean confidence with noise: {np.mean(confidences):.4f}")
    print(f"Min confidence with noise: {np.min(confidences):.4f}")
    print(f"Max confidence with noise: {np.max(confidences):.4f}")

    # Plot confidence distribution
    plt.figure(figsize=(10, 6))
    for pred in set(predictions):
        pred_conf = [r['confidence'] for r in noisy_results if r['prediction'] == pred]
        plt.hist(pred_conf, alpha=0.5, label=f'Class {pred}')

    plt.title('Model Confidence Distribution with Noise')
    plt.xlabel('Confidence')
    plt.ylabel('Count')
    plt.legend()
    plt.show()

    return noisy_results

In [885]:
confidence_results = analyze_model_confidence(10, 1, df)

Without noise:
Prediction: Class 1
Confidence: [0. 1. 0.]
Highest confidence: 1.0000

With noise:
Run 1: Class 1, confidence: 1.0000
Run 2: Class 1, confidence: 1.0000
Run 3: Class 1, confidence: 1.0000
Run 4: Class 1, confidence: 1.0000
Run 5: Class 1, confidence: 1.0000
Run 6: Class 2, confidence: 1.0000
Run 7: Class 2, confidence: 1.0000
Run 8: Class 1, confidence: 1.0000
Run 9: Class 1, confidence: 1.0000
Run 10: Class 0, confidence: 1.0000

Mean confidence with noise: 1.0000
Min confidence with noise: 1.0000
Max confidence with noise: 1.0000


In [886]:
def extract_hrv_features_robust(ecg_signal, sampling_rate=250):
    """More robust version of HRV feature extraction with additional checks."""
    try:
        # 1. Clean the ECG signal using NeuroKit
        ecg_cleaned = nk.ecg_clean(ecg_signal, sampling_rate=sampling_rate)

        # 2. R-peak detection using NeuroKit2
        _, rpeaks_info = nk.ecg_peaks(ecg_cleaned, sampling_rate=sampling_rate)
        rpeaks = rpeaks_info["ECG_R_Peaks"]  # indices of R-peaks in the signal

        # Check if we have enough R-peaks (at least 30 for reliable HRV)
        if len(rpeaks) < 30:
            print(f"Warning: Only {len(rpeaks)} R-peaks detected, HRV may be unreliable")
            if len(rpeaks) < 10:  # Not enough for even basic HRV
                return None

        # 3. Compute RR intervals (ms) from consecutive R-peak indices
        rr_intervals = np.diff(rpeaks) / sampling_rate * 1000.0

        # 4. Remove RR outliers but check how many are removed
        valid_indices = (rr_intervals >= 300) & (rr_intervals <= 2000)
        valid_rr = rr_intervals[valid_indices]

        # Check if we have enough valid intervals
        removal_percentage = (1 - len(valid_rr)/len(rr_intervals)) * 100
        if removal_percentage > 20:
            print(f"Warning: {removal_percentage:.1f}% of RR intervals were removed as outliers")
            if len(valid_rr) < 10:  # Not enough for reliable HRV
                return None

        # Continue with your existing calculations...
        # ... (rest of your function)

        # Before returning, check for extreme outliers in calculated features
        features = [mean_rr, median_rr, sdrr, rmssd, sdsd, sdrr_rmssd, hr, pnn25, pnn50,
                   sd1, sd2, kurt, skew,
                   mean_rel_rr, median_rel_rr, sdrr_rel_rr, rmssd_rel_rr, sdsd_rel_rr,
                   sdrr_rmssd_rel_rr, kurt_rel_rr, skew_rel_rr,
                   float(vlf), float(vlf_pct), float(lf), float(lf_pct), float(lf_nu),
                   float(hf), float(hf_pct), float(hf_nu),
                   float(tp), float(lf_hf), float(hf_lf),
                   float(sampen_val), float(hfd_val)]

        # Check for nan or inf values
        if any(np.isnan(f) or np.isinf(f) for f in features):
            print("Warning: NaN or Inf values detected in features")
            return None

        return features
    except Exception as e:
        print(f"Error in HRV extraction: {e}")
        return None

In [887]:
def analyze_decision_boundaries(sub, seg, df):
    """Analyze how feature changes affect classification boundaries."""
    # Get baseline features (no noise)
    base_features = generate_features_for_row(sub, seg, df, add_noise=False)
    base_hrv = base_features[:34]

    # Scale features
    base_scaled = scaler_hrv.transform(np.array(base_hrv).reshape(1, -1))
    base_pred = predict_from_hrv_features(base_hrv)

    # Feature names
    feature_names = [
        "mean_rr", "median_rr", "sdrr", "rmssd", "sdsd", "sdrr_rmssd", "hr", "pnn25", "pnn50",
        "sd1", "sd2", "kurt", "skew",
        "mean_rel_rr", "median_rel_rr", "sdrr_rel_rr", "rmssd_rel_rr", "sdsd_rel_rr",
        "sdrr_rmssd_rel_rr", "kurt_rel_rr", "skew_rel_rr",
        "vlf", "vlf_pct", "lf", "lf_pct", "lf_nu", "hf", "hf_pct", "hf_nu",
        "tp", "lf_hf", "hf_lf", "sampen_val", "hfd_val"
    ]

    # Test sensitivity to each feature
    results = []

    # For each feature, test sensitivity by perturbing just that feature
    for i, feature_name in enumerate(feature_names):
        change_pcts = []
        pred_changes = []

        # Test with different perturbation levels
        for pct in [-0.3, -0.2, -0.1, -0.05, 0.05, 0.1, 0.2, 0.3]:
            # Copy base features and modify just one
            test_features = base_hrv.copy()
            test_features[i] = base_hrv[i] * (1 + pct)

            # Get prediction
            test_pred = predict_from_hrv_features(test_features)

            # Record if prediction changed
            change_pcts.append(pct)
            pred_changes.append(1 if test_pred != base_pred else 0)

        # Calculate sensitivity score
        sensitivity = sum(pred_changes) / len(pred_changes)
        results.append({
            'feature': feature_name,
            'sensitivity': sensitivity,
            'base_value': base_hrv[i],
            'scaled_value': base_scaled[0][i]
        })

    # Sort by sensitivity
    results.sort(key=lambda x: x['sensitivity'], reverse=True)

    # Print top sensitive features
    print(f"Base prediction: Class {base_pred}")
    print("\nFeatures most likely to change prediction when perturbed:")
    for i, res in enumerate(results[:10]):
        print(f"{i+1}. {res['feature']}: {res['sensitivity']:.2f}, base value: {res['base_value']:.4f}")

    return results

In [888]:
boundary_results = analyze_decision_boundaries(10, 1, df)

Base prediction: Class 1

Features most likely to change prediction when perturbed:
1. mean_rel_rr: 0.25, base value: 0.9996
2. median_rel_rr: 0.25, base value: 1.0000
3. mean_rr: 0.00, base value: 384.4706
4. median_rr: 0.00, base value: 384.0000
5. sdrr: 0.00, base value: 4.1650
6. rmssd: 0.00, base value: 3.7523
7. sdsd: 0.00, base value: 3.7489
8. sdrr_rmssd: 0.00, base value: 1.1100
9. hr: 0.00, base value: 156.0588
10. pnn25: 0.00, base value: 0.0000


# emotion models

In [ ]:
print(trained_models_dir)

/content/drive/MyDrive/Colab Notebooks/project/trained models/


In [ ]:
happy_from_hrv_model = keras.models.load_model(trained_models_dir+"emotion models/keras/happy_from_hrv_model.keras")
sad_from_hrv_model = keras.models.load_model(trained_models_dir+"emotion models/keras/sad_from_hrv_model.keras")
anger_from_hrv_model = keras.models.load_model(trained_models_dir+"emotion models/keras/anger_from_hrv_model.keras")
neutral_from_hrv_model = keras.models.load_model(trained_models_dir+"emotion models/keras/neutral_from_hrv_model.keras")

In [ ]:
import joblib

scaler_happy = joblib.load(trained_models_dir+"emotion models/pkl/happy_from_hrv_scaler.pkl")
scaler_sad = joblib.load(trained_models_dir+"emotion models/pkl/sad_from_hrv_scaler.pkl")
scaler_anger = joblib.load(trained_models_dir+"emotion models/pkl/anger_from_hrv_scaler.pkl")
scaler_neutral = joblib.load(trained_models_dir+"emotion models/pkl/neutral_from_hrv_scaler.pkl")

In [ ]:
import numpy as np

def predict_emotions_from_hrv(sub, seg, df):
    try:
        # Extract all 34 HRV features
        features = generate_features_for_row(sub=sub, seg=seg, df=df)
        hrv_features = features[:34]

        x = np.array([hrv_features])

        # Apply appropriate scalers
        x_happy = scaler_happy.transform(x)
        x_sad = scaler_sad.transform(x)
        x_anger = scaler_anger.transform(x)
        x_neutral = scaler_neutral.transform(x)

        # Predict class probabilities
        happy_probs = happy_from_hrv_model.predict(x_happy)
        sad_probs = sad_from_hrv_model.predict(x_sad)
        anger_probs = anger_from_hrv_model.predict(x_anger)
        neutral_probs = neutral_from_hrv_model.predict(x_neutral)

        # Convert to class labels (0–4)
        happy_class = int(np.argmax(happy_probs))
        sad_class = int(np.argmax(sad_probs))
        anger_class = int(np.argmax(anger_probs))
        neutral_class = int(np.argmax(neutral_probs))


        return {
            "Happy": happy_class,
            "Sad": sad_class,
            "Anger": anger_class,
            "Neutral": neutral_class
        }

    except Exception as e:
        print(f"Emotion prediction error for sub={sub}, seg={seg}: {e}")
        return None

In [ ]:
result = predict_emotions_from_hrv(sub=10, seg=3, df=df)
print(result)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 99ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step
{'Happy': 0, 'Sad': 0, 'Anger': 0, 'Neutral': 0}


In [ ]:
# Create empty columns to store predictions
df["Happy_from_HRV"] = np.nan
df["Sad_from_HRV"] = np.nan
df["Anger_from_HRV"] = np.nan
df["Neutral_from_HRV"] = np.nan

# Loop over all rows
for idx, row in df.iterrows():
    sub = row["sub"]
    seg = row["seg"]

    emo_predictions = predict_emotions_from_hrv(sub, seg, df)

    if predictions:
        df.at[idx, "Stress_from_HRV"] = predictions["Stress_from_HRV"]
        df.at[idx, "Stress_from_Steps"] = predictions["Stress_from_Steps"]
        df.at[idx, "Stress_from_OxSleep"] = predictions["Stress_from_OxSleep"]
        df.at[idx, "Total Stress"] = predictions["Total Stress"]
    else:
        print(f"Skipped prediction for sub={sub}, seg={seg}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━

In [ ]:
df.groupby('Happy_from_HRV').size()


,0
Happy_from_HRV,


In [ ]:
df.groupby('Sad_from_HRV').size()


,0
Sad_from_HRV,


In [ ]:
df.groupby('Anger_from_HRV').size()


,0
Anger_from_HRV,


In [ ]:
df.groupby('Neutral_from_HRV').size()

,0
Neutral_from_HRV,
